# NextGEOSS
## Invoke the ITC's Multi-hazard Enhanced Risk Assessment service using JupyterLab

### Inspect the OGC WPS capabilities

In [337]:
from owslib.wps import WebProcessingService
from owslib.wps import monitorExecution
from owslib.wps import printInputOutput
from owslib.wps import BoundingBoxDataInput

In [325]:
wps_url = "http://territorial-planning.deimos.nextgeoss.terradue.com/wps/WebProcessingService"

In [326]:
wps = WebProcessingService(wps_url, verbose=False, skip_caps=True)

In [327]:
wps.getcapabilities()

In [328]:
for operation in wps.operations:
    print operation.name

GetCapabilities
DescribeProcess
Execute


In [329]:
for process in wps.processes:
    print process.identifier

com.terradue.wps_oozie.process.OozieAbstractAlgorithm


### Inspect the OGC WPS process information and inputs

In [330]:
process_id = 'com.terradue.wps_oozie.process.OozieAbstractAlgorithm'
process = wps.describeprocess(process_id)



In [331]:
process.title

'NextGEOSS7.1'

In [332]:
process.abstract

'Workflow of the territorial planning pilot for lulc classification'

In [333]:
for input in process.dataInputs:
    printInputOutput(input)
    print '***'

 identifier=Sources, title=..., abstract=..., data type=string
 Any value allowed
 Default Value: https://catalog.terradue.com/sentinel2/description 
 minOccurs=1, maxOccurs=1
***
 identifier=startdate, title=Start date/time, abstract=Start date/time (ISO8601), data type=string
 Any value allowed
 Default Value: 2015-07-26 
 minOccurs=1, maxOccurs=1
***
 identifier=enddate, title=End date/time, abstract=End date/time (ISO8601), data type=string
 Any value allowed
 Default Value: 2015-08-15 
 minOccurs=1, maxOccurs=1
***
 identifier=cc, title=Cloud Coverage, abstract=Allowed interval of cloud coverage, data type=string
 Any value allowed
 Default Value: [0,100] 
 minOccurs=1, maxOccurs=1
***
 identifier=bb, title=Bounding Box, abstract=Bounding Box, data type=BoundingBoxData
 Supported Value: urn:ogc:def:crs:EPSG:6.6:4326
 Default Value: urn:ogc:def:crs:EPSG:6.6:4326 
 minOccurs=1, maxOccurs=1
***
 identifier=aoi, title=Bounding polygon, abstract=WKT Polygon, data type=string
 Any value

### Prepare the execution request

In [351]:
bbox = BoundingBoxDataInput([-9.53,38.38,-8.47,39.08])

In [352]:
inputs = []
inputs.append(('Sources','https://catalog.terradue.com/sentinel2/description'))
inputs.append(('startdate','2015-07-26'))
inputs.append(('enddate','2015-08-15'))
inputs.append(('cc','[0,100]'))
inputs.append(('bb',bbox))
inputs.append(('aoi','MULTIPOLYGON (((-9.50982360559387 38.3997691791807,-9.52274358642492 39.069563901453,-8.48380377139078 39.0773171769051,-8.48054203826926 38.407340100023,-9.50982360559387 38.3997691791807)))'))


outputs = [('result_osd',True)]

### Execute and monitor the request

In [353]:
execution = wps.execute(process_id, inputs, outputs)

In [354]:
monitorExecution(execution)

In [355]:
execution.isSucceded()

True

### Access the results' OpenSearch Description (osd)

In [356]:
for output in execution.processOutputs:
    print(output.identifier)

result_osd


In [357]:
results_osd = execution.processOutputs[0].reference

In [358]:
print results_osd

http://territorial-planning.deimos.nextgeoss.terradue.com/wps/RetrieveResultServlet?id=bd6494e7-79c3-449e-9562-d9721a9ca130result_osd.3a36b136-bcd5-4e3b-987b-7bab9ebdd8b1


In [359]:
import requests
r = requests.get(results_osd)
r.text

u'<?xml version=\'1.0\' encoding=\'UTF-8\'?><ns:Reference xmlns:ns="http://www.opengis.net/wps/1.0.0" href="http://territorial-planning.deimos.nextgeoss.terradue.com/sbws/wps/workflow/0000010-190709165332500-oozie-oozi-W/results/description" method="GET" mimeType="application/opensearchdescription+xml"/>'

In [360]:
import xml.etree.ElementTree as ET
root = ET.fromstring(r.text)
root.attrib['href']

'http://territorial-planning.deimos.nextgeoss.terradue.com/sbws/wps/workflow/0000010-190709165332500-oozie-oozi-W/results/description'

### Perform an OpenSearch query for the results enclosures

In [361]:
import cioppy

In [362]:
ciop = cioppy.Cioppy()

In [363]:
search = ciop.search(end_point = root.attrib['href'],
                     params = [],
                     output_fields='enclosure')

In [364]:
for result in search:
    print result['enclosure']

http://territorial-planning.deimos.nextgeoss.terradue.com/data/run/workflow/0000010-190709165332500-oozie-oozi-W/_results/L2A_T29SMC_A000506_20150728T113618_LULC.zip
http://territorial-planning.deimos.nextgeoss.terradue.com/data/run/workflow/0000010-190709165332500-oozie-oozi-W/_results/L2A_T29SMD_A000506_20150728T113618_LULC.zip
http://territorial-planning.deimos.nextgeoss.terradue.com/data/run/workflow/0000010-190709165332500-oozie-oozi-W/_results/L2A_T29SNC_A000506_20150728T113618_LULC.zip
http://territorial-planning.deimos.nextgeoss.terradue.com/data/run/workflow/0000010-190709165332500-oozie-oozi-W/_results/L2A_T29SND_A000506_20150728T113618_LULC.zip


### Download an output file

The following section can be leveraged when published results are available.

In [24]:
#search[0]['enclosure']

In [25]:
#r = requests.get(search[0]['enclosure'],stream=True)

In [26]:
#with open('output', 'wb') as fd:
#    for chunk in r.iter_content(chunk_size=128):
#        fd.write(chunk)